In [71]:
# imports
import os
import numpy as np
import pandas as pd
import kagglehub
from IPython.display import display
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [72]:
# Download latest version
path = kagglehub.dataset_download("austinreese/craigslist-carstrucks-data")

# print("Path to dataset files:", path)
# print(os.listdir(path))

# df = pd.read_csv(os.path.join(path, "vehicles.csv"))

# display(df.head(5))

In [ ]:
# read the dataframe
df_read = pd.read_csv(os.path.join(path, "vehicles.csv"))
print(df_read.dtypes)

In [93]:
# trim and preprocess
df = df_read[['price',
                   'year',
                   'manufacturer',
                   'model',
                   'condition',
                   'fuel',
                   'odometer',
                   'drive',
                   'type',
                   'paint_color',
                   'state',
                   'lat',
                   'long']].dropna()

# trim low price and mileage
df = df[df['price'] >= 1000]
df = df[df['price'] <= 40000]
df = df[df["odometer"] >= 5000]
df = df[df["odometer"] <= 300000]

# replace year with age
current_year = 2025
df["age"] = current_year - df["year"]
df.drop('year', axis=1, inplace=True)

# split into model and trim, fill empties with unknown
split = df["model"].str.split()
df["model"] = split.str[:2].str.join(" ")
df["trim"] = split.str[2:].str.join(" ").replace("", "Unknown")

# Define features and target
features_X = df[['age',
                   'manufacturer',
                   'model',
                   'trim',
                   'condition',
                   'fuel',
                   'odometer',
                   'drive',
                   'type',
                   'paint_color',
                   'state',
                   'lat',
                   'long']]
target_y = df['price']
print(features_X.columns)
print(df_read.columns)

Index(['age', 'manufacturer', 'model', 'trim', 'condition', 'fuel', 'odometer',
       'drive', 'type', 'paint_color', 'state', 'lat', 'long'],
      dtype='object')
Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')


In [75]:
print(df.dtypes)

price             int64
manufacturer     object
model            object
condition        object
fuel             object
odometer        float64
drive            object
type             object
paint_color      object
state            object
age             float64
trim             object
dtype: object


In [87]:
#split into train and test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(features_X, target_y, test_size=0.2)

In [88]:
# define categorical and numerical columns
cat_cols = X_train.select_dtypes(include="object").columns.tolist()
num_cols = X_train.select_dtypes(exclude="object").columns.tolist()

In [89]:
# one-hot encode the categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('numeric','passthrough', num_cols)
    ]
)

In [79]:
# # build pipeline with the linear regression model
# model = Pipeline(steps=[
#     ('preprocess', preprocessor),
#     ('regressor', LinearRegression())
# ])

In [90]:
# build pipeline with the random forest model
model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=100,
        max_depth=20,
        random_state=42,
        n_jobs=-1))
])

In [91]:
# Try log values of y
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)
# fit the model
model.fit(X_train, y_train_log)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['manufacturer', 'model',
                                                   'trim', 'condition', 'fuel',
                                                   'drive', 'type',
                                                   'paint_color', 'state']),
                                                 ('numeric', 'passthrough',
                                                  ['age', 'odometer', 'lat',
                                                   'long'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=20, n_jobs=-1,
                                       random_state=42))])

In [94]:
# predict and report error
y_pred_log = model.predict(X_test)
y_pred = np.exp(y_pred_log)
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ", mae)
# current MAE: 2200
print(sklearn.__version__)

MAE:  2184.557838200449
1.6.1


In [83]:
# Compare error
comparison = pd.DataFrame()
comparison["true_price"] = y_test.values
comparison["predicted_price"] = y_pred
comparison["error"] = comparison["predicted_price"] - comparison["true_price"]
comparison["absolute % error"] = abs(comparison["error"] / comparison["true_price"] * 100)

print(comparison.describe())

         true_price  predicted_price         error  absolute % error
count  24751.000000     24751.000000  24751.000000      2.475100e+04
mean   15553.295342     15090.145660   -463.149681      2.269428e+01
std    10390.247622      9702.949119   3530.988087      5.199364e+01
min     1000.000000      1000.000000 -29065.794631      1.213064e-14
25%     6990.000000      7047.965376  -1605.265419      4.138025e+00
50%    12995.000000     12372.017296    -81.793533      1.192531e+01
75%    22981.500000     21466.848207   1121.776632      2.631489e+01
max    40000.000000     39960.295952  25911.300969      2.578648e+03


In [84]:
my_car = pd.DataFrame([{
    'age': 20.0,
                   'manufacturer': 'jeep',
                   'model': 'grand cherokee',
                   'trim': 'laredo',
                   'condition': 'good',
                   'fuel': 'gas',
                   'odometer': 170000,
                   'drive': '4wd',
                   'type': 'SUV',
                   'paint_color': 'red',
                   'state': 'ct'
}])

norah_car = pd.DataFrame([{
    'age': 13.0,
                   'manufacturer': 'honda',
                   'model': 'crv',
                   'trim': "",
                   'condition': 'fair',
                   'fuel': 'gas',
                   'odometer': 264000,
                   'drive': 'fwd',
                   'type': 'SUV',
                   'paint_color': 'white',
                   'state': 'tx'
}])

jodi = pd.DataFrame([{
    'age': 4.0,
                   'manufacturer': 'hyundai',
                   'model': 'palisade',
                   'trim': 'calligraphy',
                   'condition': 'excellent',
                   'fuel': 'gas',
                   'odometer': 70000,
                   'drive': '4wd',
                   'type': 'SUV',
                   'paint_color': 'black',
                   'state': 'ct'
}])

tesla = pd.DataFrame([{
    'age': 3,
                   'manufacturer': 'tesla',
                   'model': 'model y',
                   'trim': '',
                   'condition': 'excellent',
                   'fuel': 'electric',
                   'odometer': 30000,
                   'drive': '4wd',
                   'type': 'SUV',
                   'paint_color': 'red',
                   'state': 'ct'
}])

my_car_value = np.exp(model.predict(jodi))

print(f"Estimated value of car: ${my_car_value[0]:.2f}")

print(f"Conditions: {df["condition"].unique()}")
print(f"Drives: {df["drive"].unique()}")
print(f"States: {df["state"].unique()}")
print(f"Colors: {df["paint_color"].unique()}")
print(f"Types: {df["type"].unique()}")
print(f"Fuels: {df["fuel"].unique()}")

Estimated value of car: $23494.35
Conditions: ['excellent' 'good' 'new' 'fair' 'like new' 'salvage']
Drives: ['rwd' '4wd' 'fwd']
States: ['al' 'ak' 'az' 'ar' 'ca' 'co' 'ct' 'dc' 'de' 'fl' 'ga' 'hi' 'id' 'il'
 'in' 'ia' 'ks' 'ky' 'la' 'me' 'md' 'ma' 'mi' 'mn' 'ms' 'mo' 'mt' 'nc'
 'ne' 'nv' 'nj' 'nm' 'ny' 'nh' 'nd' 'oh' 'ok' 'or' 'pa' 'ri' 'sc' 'sd'
 'tn' 'tx' 'ut' 'vt' 'va' 'wa' 'wv' 'wi' 'wy']
Colors: ['black' 'silver' 'grey' 'red' 'blue' 'white' 'brown' 'yellow' 'green'
 'custom' 'orange' 'purple']
Types: ['truck' 'pickup' 'other' 'coupe' 'SUV' 'hatchback' 'mini-van' 'sedan'
 'offroad' 'convertible' 'wagon' 'van' 'bus']
Fuels: ['gas' 'other' 'diesel' 'hybrid' 'electric']
